In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from collections import defaultdict
from gensim.models import Phrases
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [1]:
import preprocess_text as pt

#import our function to import data and clean it 
train_df,val_df = pt.suicidal_intent_data_load(test_dataset = False,remove_stopwords = False,standardization=True)
train_df.head()

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/
Hi


,target,text
1370626,0,lmao that funni but true
82069,1,i miss u and twitter so much my bfs phone is w...
1489496,0,dalilah got me a francisco rodriguez angel bas...
1424058,0,had a good night at rutherglen half my famili ...
868896,0,meanwhil on other news ryan gigg just score


In [4]:
new_df = pd.concat([train_df,val_df])

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

# create a CountVectorizer object
count_vec = CountVectorizer()

# fit 
count_vec.fit(new_df.text)
#transform
x_train_cvec = count_vec.transform(train_df.text)
#x_test_cvec  = count_vec.fit_transform(test_df.text)
val_df_cvec  = count_vec.transform(val_df.text)

#new_df_cvec = count_vec.fit_transform(new_df.text)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

# create a TF-IDF Vectorizer object
tfidf_vec = TfidfVectorizer(ngram_range=(1,2),max_features=3891472)
#fit
#tfidf_vec.fit(train_df.text)
tfidf_vec.fit(new_df.text)

TfidfVectorizer(max_features=3891472, ngram_range=(1, 2))

In [7]:
#transform
x_train_tfidf = tfidf_vec.transform(train_df.text)
#x_test_tfidf  = tfidf_vec.transform(test_df.text)
val_df_tfidf  = tfidf_vec.transform(val_df.text)

##### Word2Vec

In [8]:
import gensim

# Tokenize the sentences in the train DataFrame
sentences = train_df['text'].apply(lambda x: x.split()).tolist()
# Train your Word2Vec model
model = gensim.models.Word2Vec(sentences, min_count=1)

In [9]:
#Define generate embeddings function
def generate_embeddings(text, model):
    words = text.split()
    embeddings = []
    for word in words:
        if word in  model.wv.key_to_index:
            embeddings.append(model.wv.get_vector(word, norm=True))
    if len(embeddings) > 0:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(model.vector_size)

In [10]:
train_df['embeddings'] = train_df['text'].apply(lambda x: generate_embeddings(x, model))
val_df['embeddings'] = val_df['text'].apply(lambda x: generate_embeddings(x, model))

In [13]:
X_train_w2vec = np.stack(train_df['embeddings'].values)
y_train_w2vec = train_df['target'].values

val_w2vec = np.stack(val_df['embeddings'].values)
y_val_w2vec = val_df['target'].values

### Modeling

<b> XGBoost </b>

In [11]:
from xgboost import XGBClassifier
from sklearn.metrics import (precision_score, 
                             recall_score, 
                             f1_score, 
                             classification_report,
                             accuracy_score)

XGBoost with TF IDF

In [12]:
XGBReg = XGBClassifier(max_depth=10,n_jobs=6)
XGBReg.fit(x_train_tfidf, train_df.target)
y_pred = XGBReg.predict(val_df_tfidf)
accuracy = accuracy_score(val_df.target, y_pred)
precision = precision_score(val_df.target, y_pred)
recall = recall_score(val_df.target, y_pred)
f1 = f1_score(val_df.target, y_pred)

print(f"Model: Logistic Regression on TF-IDF")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 score: {f1:.2f}")

Model: Logistic Regression on TF-IDF
Accuracy: 0.78
Precision: 0.81
Recall: 0.75
F1 score: 0.77


XGBoost with CountVectorizer

In [14]:
XGBReg2 = XGBClassifier(max_depth=10,n_jobs=6)
XGBReg2.fit(x_train_cvec, train_df.target)
y_pred = XGBReg2.predict(val_df_cvec)

accuracy = accuracy_score(val_df.target, y_pred)
precision = precision_score(val_df.target, y_pred)
recall = recall_score(val_df.target, y_pred)
f1 = f1_score(val_df.target, y_pred)

print(f"Model: Logistic Regression on CountVectorizer")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 score: {f1:.2f}")

Model: Logistic Regression on CountVectorizer
Accuracy: 0.78
Precision: 0.80
Recall: 0.74
F1 score: 0.77


XGBoost with Word2Vec

In [15]:
XGBReg3 = XGBClassifier(max_depth=10,n_jobs=6)
XGBReg3.fit(X_train_w2vec,y_train_w2vec )
y_pred = XGBReg3.predict(val_w2vec)

accuracy = accuracy_score(y_val_w2vec, y_pred)
precision = precision_score(y_val_w2vec, y_pred)
recall = recall_score(y_val_w2vec, y_pred)
f1 = f1_score(y_val_w2vec, y_pred)

print(f"Model: Logistic Regression on Word2Vec")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 score: {f1:.2f}")

Model: Logistic Regression on Word2Vec
Accuracy: 0.77
Precision: 0.77
Recall: 0.78
F1 score: 0.77
